In [ ]:
!nvidia-smi

Sat Sep 26 21:29:17 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    34W / 250W |   2083MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
!pip install transformers

     |████████████████████████████████| 1.0MB 2.8MB/s 
     |████████████████████████████████| 1.1MB 17.9MB/s 
     |████████████████████████████████| 3.0MB 20.9MB/s 
     |████████████████████████████████| 890kB 39.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=ba15c93df2e2d2d8910252fa56c41bcb772d92b7920aad0cf91d8704a642c81f
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
!pip install normalise
!pip install pyenchant
!apt-get install libenchant1c2a

     |████████████████████████████████| 15.7MB 298kB/s 
     |████████████████████████████████| 61kB 1.8MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  aspell aspell-en dictionaries-common emacsen-common enchant hunspell-en-us
  libaspell15 libhunspell-1.6-0 libtext-iconv-perl
Suggested packages:
  aspell-doc spellutils wordlist hunspell openoffice.org-hunspell
  | openoffice.org-core libenchant-voikko
The following NEW packages will be installed:
  aspell aspell-en dictionaries-common emacsen-common enchant hunspell-en-us
  libaspell15 libenchant1c2a libhunspell-1.6-0 libtext-iconv-perl
0 upgraded, 10 newly installed, 0 to remove and 11 not upgraded.
Need to get 1,310 kB of archives.
After this operation, 5,353 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libtext-iconv-perl amd64 1.7-5build6 [13.0 kB]
Get:2 http://archive.ub

In [3]:
import torch
import pandas as pd
import numpy as np
import random
from sklearn.metrics import f1_score
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
import torch.nn as nn
from torch.utils.data import TensorDataset
from transformers import BertModel
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, Dataset, WeightedRandomSampler
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.utils import shuffle
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
from tqdm.notebook import tqdm
import sys
import re
import nltk
from scipy import stats
import warnings
warnings.filterwarnings("ignore")

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
%%javascript
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}setInterval(ClickConnect,60000)

<IPython.core.display.Javascript object>

In [4]:
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100 

In [5]:
folder_path = '/content/drive/My Drive/chatbot/'
zip_path = '/content/drive/My Drive/chatbot/liar_dataset.zip'

# **Data Preprocessing-Liar Plus**

In [ ]:
# with zipfile.ZipFile(zip_path, 'r') as zip:
#   zip.extractall(folder_path)

In [ ]:
train_path = '/content/drive/My Drive/chatbot/archive/LIAR-PLUS-master/dataset/train2.tsv'
val_path = '/content/drive/My Drive/chatbot/archive/LIAR-PLUS-master/dataset/val2.tsv'
test_path = '/content/drive/My Drive/chatbot/archive/LIAR-PLUS-master/dataset/test2.tsv'

In [ ]:
df_train = pd.read_csv(train_path, names = ["id", "label", "statement", "subject", "speaker", "job", "state", "party",
                                            "barely-true", "false", "half-true",
                                            "mostly-true", "pants-fire", "venue","justification"], sep='\t').reset_index(drop=True)
df_train

,id,label,statement,subject,speaker,job,state,party,barely-true,false,half-true,mostly-true,pants-fire,venue,justification
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer,That's a premise that he fails to back up. Ann...
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.,"Surovell said the decline of coal ""started whe..."
2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver,Obama said he would have voted against the ame...
3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release,The release may have a point that Mikulskis co...
4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN,"Crist said that the economic ""turnaround start..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10235,5473.json,mostly-true,There are a larger number of shark attacks in ...,"animals,elections",aclu-florida,NaN,Florida,none,0.0,1.0,1.0,1.0,0.0,"interview on ""The Colbert Report""",They compounded their error by combining full ...
10236,3408.json,mostly-true,Democrats have now become the party of the [At...,elections,alan-powell,NaN,Georgia,republican,0.0,0.0,0.0,1.0,0.0,an interview,"Romney said that ""Obamacare means that for up..."
10237,3959.json,half-true,Says an alternative to Social Security that op...,"retirement,social-security",herman-cain,NaN,Georgia,republican,4.0,11.0,5.0,3.0,3.0,a Republican presidential debate,But that it leaves out important details and t...
10238,2253.json,false,On lifting the U.S. Cuban embargo and allowing...,"florida,foreign-policy",jeff-greene,NaN,Florida,democrat,3.0,1.0,3.0,0.0,0.0,a televised debate on Miami's WPLG-10 against ...,"We checked the research and, quite frankly, fi..."


In [ ]:
df_val = pd.read_csv(val_path, names = ["id", "label", "statement", "subject", "speaker", "job", "state", "party",
                                            "barely-true", "false", "half-true",
                                            "mostly-true", "pants-fire", "venue","justification"], sep='\t').reset_index(drop=True)
df_val

,id,label,statement,subject,speaker,job,state,party,barely-true,false,half-true,mostly-true,pants-fire,venue,justification
0,12134.json,barely-true,We have less Americans working now than in the...,"economy,jobs",vicky-hartzler,U.S. Representative,Missouri,republican,1,0,1,0,0,an interview with ABC17 News,"However, Hartzler was talking about the entire..."
1,238.json,pants-fire,"When Obama was sworn into office, he DID NOT u...","obama-birth-certificate,religion",chain-email,NaN,NaN,none,11,43,8,5,105,NaN,Ellison used a Koran that once belonged to Tho...
2,7891.json,false,Says Having organizations parading as being so...,"campaign-finance,congress,taxes",earl-blumenauer,U.S. representative,Oregon,democrat,0,1,1,1,0,a U.S. Ways and Means hearing,"However, we have two professors who say the la..."
3,8169.json,half-true,Says nearly half of Oregons children are poor.,poverty,jim-francesconi,Member of the State Board of Higher Education,Oregon,none,0,1,1,1,0,an opinion article,"In fact, if you use federal definitions for po..."
4,929.json,half-true,On attacks by Republicans that various program...,"economy,stimulus",barack-obama,President,Illinois,democrat,70,71,160,163,9,interview with CBS News,Obama's point is that some perspective is in o...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1279,3419.json,half-true,"For the first time in more than a decade, impo...","energy,oil-spill,trade",barack-obama,President,Illinois,democrat,70,71,160,163,9,a press conference,"In 2009, 17 percent of the U. S. 's oil import..."
1280,12548.json,mostly-true,Says Donald Trump has bankrupted his companies...,candidates-biography,hillary-clinton,Presidential candidate,New York,democrat,40,29,69,76,7,a speech on the economy,"Clinton said, Trump has ""bankrupted his compan..."
1281,401.json,true,"John McCain and George Bush have ""absolutely n...",health-care,campaign-defend-america,NaN,"Washington, D.C.",none,0,1,0,2,0,a television ad,"""I don't think that there should be a mandate ..."
1282,1055.json,false,A new poll shows 62 percent support the presid...,health-care,americans-united-change,NaN,NaN,none,1,4,4,1,0,an Internet ad.,But the poll doesn't say that. Several days af...


In [ ]:
df_test = pd.read_csv(test_path, names = ["id", "label", "statement", "subject", "speaker", "job", "state", "party",
                                            "barely-true", "false", "half-true",
                                            "mostly-true", "pants-fire", "venue","justification"], sep='\t').reset_index(drop=True)
df_test

,id,label,statement,subject,speaker,job,state,party,barely-true,false,half-true,mostly-true,pants-fire,venue,justification
0,11972.json,true,Building a wall on the U.S.-Mexico border will...,immigration,rick-perry,Governor,Texas,republican,30,30,42,23,18,Radio interview,"Meantime, engineering experts agree the wall w..."
1,11685.json,false,Wisconsin is on pace to double the number of l...,jobs,katrina-shankland,State representative,Wisconsin,democrat,2,1,0,0,0,a news conference,She cited layoff notices received by the state...
2,11096.json,false,Says John McCain has done nothing to help the ...,"military,veterans,voting-record",donald-trump,President-Elect,New York,republican,63,114,51,37,61,comments on ABC's This Week.,"Trump said that McCain ""has done nothing to he..."
3,5209.json,half-true,Suzanne Bonamici supports a plan that will cut...,"medicare,message-machine-2012,campaign-adverti...",rob-cornilles,consultant,Oregon,republican,1,1,3,1,1,a radio show,"But spending still goes up. In addition, many ..."
4,9524.json,pants-fire,When asked by a reporter whether hes at the ce...,"campaign-finance,legal-issues,campaign-adverti...",state-democratic-party-wisconsin,NaN,Wisconsin,democrat,5,7,2,2,7,a web video,Our rating A Democratic Party web video making...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1262,7334.json,half-true,Says his budget provides the highest state fun...,education,rick-scott,Governor,Florida,republican,28,23,38,34,7,a news conference,LeMieux didn't compare Rubio and Obama on an i...
1263,9788.json,barely-true,Ive been here almost every day.,"civil-rights,crime,criminal-justice",jay-nixon,Governor,Missouri,democrat,2,0,0,1,0,"on ABC's ""This Week""","After making his pledge, Obama said the budget..."
1264,10710.json,barely-true,"In the early 1980s, Sen. Edward Kennedy secret...","bipartisanship,congress,foreign-policy,history",mackubin-thomas-owens,"senior fellow, Foreign Policy Research Institute",Rhode Island,columnist,1,0,0,0,0,a commentary in The Providence Journal,Former President Clinton said government got t...
1265,3186.json,barely-true,Says an EPA permit languished under Strickland...,"environment,government-efficiency",john-kasich,"Governor of Ohio as of Jan. 10, 2011",Ohio,republican,9,8,10,18,3,a news conference,Points of Light has a unique mission carved ou...


In [ ]:
labels = {'pants-fire':0, 'false':0, 'barely-true':0, 'half-true':0, 'mostly-true':1, 'true':1}


In [ ]:
df_train['target'] = df_train.label.replace(labels) 
df_val['target'] = df_val.label.replace(labels)
df_test['target'] = df_test.label.replace(labels)

In [ ]:
df_train = df_train[['statement','justification','target']].reset_index(drop=True)
df_val = df_val[['statement','justification','target']].reset_index(drop=True)
df_test = df_test[['statement','justification','target']].reset_index(drop=True)

In [ ]:
df_train.to_csv(folder_path + 'liar+train.csv',index=False)
df_val.to_csv(folder_path + 'liar+val.csv',index=False)
df_test.to_csv(folder_path + 'liar+test.csv',index=False)

In [ ]:
df_train.rename(columns = {'statement':'text_a','justification':'text_b','target':'label'}, inplace = True) 
df_val.rename(columns = {'statement':'text_a','justification':'text_b','target':'label'}, inplace = True) 
df_test.rename(columns = {'statement':'text_a','justification':'text_b','target':'label'}, inplace = True) 

In [ ]:
df_test

,text_a,text_b,label
0,Building a wall on the U.S.-Mexico border will...,"Meantime, engineering experts agree the wall w...",1
1,Wisconsin is on pace to double the number of l...,She cited layoff notices received by the state...,0
2,Says John McCain has done nothing to help the ...,"Trump said that McCain ""has done nothing to he...",0
3,Suzanne Bonamici supports a plan that will cut...,"But spending still goes up. In addition, many ...",0
4,When asked by a reporter whether hes at the ce...,Our rating A Democratic Party web video making...,0
...,...,...,...
1262,Says his budget provides the highest state fun...,LeMieux didn't compare Rubio and Obama on an i...,0
1263,Ive been here almost every day.,"After making his pledge, Obama said the budget...",0
1264,"In the early 1980s, Sen. Edward Kennedy secret...",Former President Clinton said government got t...,0
1265,Says an EPA permit languished under Strickland...,Points of Light has a unique mission carved ou...,0


# **Preprocessing and adding FNC-1 Data**

In [ ]:
!git clone https://github.com/UKPLab/coling2018_fake-news-challenge.git

Cloning into 'coling2018_fake-news-challenge'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 271 (delta 2), reused 0 (delta 0), pack-reused 265
Receiving objects: 100% (271/271), 62.59 MiB | 19.05 MiB/s, done.
Resolving deltas: 100% (90/90), done.
Checking out files: 100% (197/197), done.


In [ ]:
!wget https://github.com/UKPLab/coling2018_fake-news-challenge/blob/master/data/fnc-1/corpora/FNC_ARC/combined_stances_train.csv
!wget https://github.com/UKPLab/coling2018_fake-news-challenge/blob/master/data/fnc-1/corpora/FNC_ARC/combined_bodies_train.csv
!wget https://github.com/UKPLab/coling2018_fake-news-challenge/blob/master/data/fnc-1/corpora/FNC_ARC/combined_stances_test.csv
!wget https://github.com/UKPLab/coling2018_fake-news-challenge/blob/master/data/fnc-1/corpora/FNC_ARC/combined_bodies_test.csv

--2020-09-26 12:07:07--  https://github.com/UKPLab/coling2018_fake-news-challenge/blob/master/data/fnc-1/corpora/FNC_ARC/combined_stances_train.csv
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘combined_stances_train.csv’

combined_stances_tr     [ <=>                ]  77.92K  --.-KB/s    in 0.03s   

2020-09-26 12:07:07 (2.78 MB/s) - ‘combined_stances_train.csv’ saved [79790]

--2020-09-26 12:07:07--  https://github.com/UKPLab/coling2018_fake-news-challenge/blob/master/data/fnc-1/corpora/FNC_ARC/combined_bodies_train.csv
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘combined_bodies_train.csv’

combined_bodies_tra     [ <=>                ]  77.90K  --.-KB/s   

In [ ]:
import os
import csv
import pandas as pd
from tqdm import tqdm

from sklearn.model_selection import train_test_split

def fnc(path_headlines, path_bodies):

    map = {'disagree':0,'agree': 1 , 'discuss':2, 'unrelated':3}

    with open(path_bodies, encoding='utf_8') as fb:  # Body ID,articleBody
        body_dict = {}
        lines_b = csv.reader(fb)
        for i, line in enumerate(tqdm(list(lines_b), ncols=80, leave=False)):
            if i > 0:
                body_id = int(line[0].strip())
                body_dict[body_id] = line[1]

    with open(path_headlines, encoding='utf_8') as fh: # Headline,Body ID,Stance
        lines_h = csv.reader(fh)
        h = []
        b = []
        l = []
        for i, line in enumerate(tqdm(list(lines_h), ncols=80, leave=False)):
            if i > 0:
                body_id = int(line[1].strip())
                label = line[2].strip()
                if label in map and body_id in body_dict:
                    h.append(line[0])
                    l.append(map[line[2]])
                    b.append(body_dict[body_id])
    return h, b, l

headlines, bodies, labels = fnc(
    '/content/coling2018_fake-news-challenge/data/fnc-1/corpora/FNC_ARC/combined_stances_train.csv',
    '/content/coling2018_fake-news-challenge/data/fnc-1/corpora/FNC_ARC/combined_bodies_train.csv'
)

list_of_tuples = list(zip(headlines, bodies, labels))
df = pd.DataFrame(list_of_tuples, columns=['text_a', 'text_b', 'label'])
train_df, val_df = train_test_split(df,test_size=0.20,random_state=42)
labels_val = pd.Series(val_df['label']).to_numpy()


In [ ]:
headlines, bodies, labels = fnc(
    '/content/coling2018_fake-news-challenge/data/fnc-1/corpora/FNC_ARC/combined_stances_test.csv',
    '/content/coling2018_fake-news-challenge/data/fnc-1/corpora/FNC_ARC/combined_bodies_test.csv'
)

list_of_tuples = list(zip(headlines, bodies, labels))
test_df = pd.DataFrame(list_of_tuples, columns=['text_a', 'text_b', 'label'])
labels_test = pd.Series(test_df['label']).to_numpy()

In [ ]:
df_train_final = train_df.append(df_train)
df_val_final = val_df.append(df_val)
df_test_final = test_df.append(df_test)

In [ ]:
df_val_final.label.value_counts(),df_val.label.value_counts(),val_df.label.value_counts()

(3    9454
 2    1956
 1    1419
 0    1296
 Name: label, dtype: int64, 0    864
 1    420
 Name: label, dtype: int64, 3    9454
 2    1956
 1     999
 0     432
 Name: label, dtype: int64)

In [ ]:
df_test_final.label.value_counts(),df_test.label.value_counts(),test_df.label.value_counts()

(3    21023
 2     4643
 1     2686
 0     1887
 Name: label, dtype: int64, 0    818
 1    449
 Name: label, dtype: int64, 3    21023
 2     4643
 1     2237
 0     1069
 Name: label, dtype: int64)

In [ ]:
df_train_final.label.value_counts(),df_train.label.value_counts(),train_df.label.value_counts()

(3    37761
 0     8412
 2     7857
 1     7574
 Name: label, dtype: int64, 0    6602
 1    3638
 Name: label, dtype: int64, 3    37761
 2     7857
 1     3936
 0     1810
 Name: label, dtype: int64)

In [ ]:
df_train_final.to_csv(folder_path + 'df_train.csv',index=False)
df_val_final.to_csv(folder_path + 'df_val.csv',index=False)
df_test_final.to_csv(folder_path + 'df_test.csv',index=False)

# **Tokenisation and dataloader**

In [6]:
df_train = pd.read_csv(folder_path + 'df_train.csv').reset_index(drop=True)
df_train

FileNotFoundError: ignored

In [ ]:
df_val = pd.read_csv(folder_path + 'df_val.csv').reset_index(drop=True)
df_val

,text_a,text_b,label
0,Do not drop sports program in undergraduate sc...,If medicine were still at the level of the 19t...,3
1,IPhone 6 users are tearing their hair out,(Reuters) - First there was bendgate... now br...,2
2,MCDONALD’S WILL STOP SERVING OVERWEIGHT CUSTOM...,While Apple announced that the base model of i...,3
3,The bar is not too low to get into law school,When I observe that admission standards and co...,3
4,Rim should be raised,Immigration doesn't even enter the top 10 impo...,3
...,...,...,...
14120,"For the first time in more than a decade, impo...","In 2009, 17 percent of the U. S. 's oil import...",0
14121,Says Donald Trump has bankrupted his companies...,"Clinton said, Trump has ""bankrupted his compan...",1
14122,"John McCain and George Bush have ""absolutely n...","""I don't think that there should be a mandate ...",1
14123,A new poll shows 62 percent support the presid...,But the poll doesn't say that. Several days af...,0


In [ ]:
df_test = pd.read_csv(folder_path + 'df_test.csv').reset_index(drop=True)
df_test

,text_a,text_b,label
0,Ferguson riots: Pregnant woman loses eye after...,A RESPECTED senior French police officer inves...,3
1,Crazy Conservatives Are Sure a Gitmo Detainee ...,Dave Morin's social networking company Path is...,3
2,A Russian Guy Says His Justin Bieber Ringtone ...,A bereaved Afghan mother took revenge on the T...,3
3,"Zombie Cat: Buried Kitty Believed Dead, Meows ...",Hewlett-Packard is officially splitting in two...,3
4,Argentina's President Adopts Boy to End Werewo...,An airline passenger headed to Dallas was remo...,3
...,...,...,...
30234,Says his budget provides the highest state fun...,LeMieux didn't compare Rubio and Obama on an i...,0
30235,Ive been here almost every day.,"After making his pledge, Obama said the budget...",0
30236,"In the early 1980s, Sen. Edward Kennedy secret...",Former President Clinton said government got t...,0
30237,Says an EPA permit languished under Strickland...,Points of Light has a unique mission carved ou...,0


In [ ]:
print(df_test.text_b.iloc[30234])
print('-------------------------------XX-----------')
df_test.text_a.iloc[30234]


LeMieux didn't compare Rubio and Obama on an issue such as those listed at the start of the ad -- he said they both used a familiar campaign tactic, throwing "bombs" about something they didn't have to vote on themselves. The ad provides no explanation for how he compared the two politicians and neglects to note that LeMieux supported Rubio's campaign once Crist left the GOP.
-------------------------------XX-----------


'Says his budget provides the highest state funding level in history for education.'

In [ ]:
labels = {'disagree':0,'agree': 1 , 'discuss':2, 'unrelated':3}
num_classes = len(labels)
num_classes

4

In [ ]:
class BERTDataset(Dataset):

  def __init__(self,claim ,ground_truth , labels, tokenizer, max_len = 256):
    self.claim = claim
    self.ground_truth = ground_truth
    self.labels = labels
    self.tokenizer = tokenizer
    self.max_len = max_len
    
  def __len__(self):
    return len(self.labels)
  
  def __getitem__(self, item):
    
    claim = str(self.claim[item])
    gt = str(self.ground_truth[item])
    label = self.labels[item]

    encoded_data = self.tokenizer.encode_plus(
        gt,
        claim,
        add_special_tokens=True,   
        max_length=self.max_len,
        truncation = True
    )

    ids = encoded_data['input_ids']
    token_type_ids = encoded_data['token_type_ids']
    mask = encoded_data['attention_mask']

    padding_len = self.max_len - len(ids)
    ids = ids + ([0]*padding_len)
    token_type_ids = token_type_ids + ([0]*padding_len)
    mask = mask + ([0]*padding_len)

    return {
        "ids" : torch.tensor(ids, dtype=torch.long),
        "token_type_ids" : torch.tensor(token_type_ids, dtype=torch.long),
        "mask": torch.tensor(mask, dtype=torch.long),
        "targets": torch.tensor(label, dtype=torch.float)
    }


In [ ]:
class BERTModel(nn.Module):
  
  def __init__(self):
    super(BERTModel,self).__init__()
    self.bert = BertModel.from_pretrained("bert-base-uncased")
    self.dropout = nn.Dropout(0.2)
    self.out = nn.Linear(768,6)
  
  def forward(self,ids,mask,token_type_ids):
    _, o2 = self.bert(ids, attention_mask=mask,token_type_ids=token_type_ids)
    bo = self.dropout(o2)
    return self.out(bo)


In [ ]:
label_dict = labels
label_dict

{'agree': 1, 'disagree': 0, 'discuss': 2, 'unrelated': 3}

In [ ]:
def loss_func(outputs, targets):
  return nn.BCEWithLogitsLoss()(outputs,targets)

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, targets):
    
    label_orig = labels
    label_dict_inverse = {v: k for k, v in label_dict.items()}

    F1_score = f1_score_func(preds,targets)
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = targets.flatten()
    correct = len(preds_flat[preds_flat == labels_flat])
    total = len(labels_flat)
    accuracy = correct/total
    
    print(f'Accuracy: {accuracy}')
    print(f'F1 Score: {F1_score}')
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        acc = len(y_preds[y_preds==label])/len(y_true)
        print(f'Class: { label_dict_inverse[label] }')
        print(f'Class accuracy: {len(y_preds[y_preds==label])}/{len(y_true)},{acc}\n')

In [ ]:
def train_fn(model,dataloader_train,optimizer,schedular,device,epoch):
  
  model.train()
  loss_train_total = 0
  progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)

  for batch in progress_bar:

    model.zero_grad()
    ids = batch['ids']
    mask = batch['mask']
    token_type_ids = batch['token_type_ids']
    labels = batch['targets']
    
    ids = ids.to(device, dtype=torch.long)
    mask = mask.to(device, dtype=torch.long)
    token_type_ids = token_type_ids.to(device, dtype=torch.long)
    labels = labels.to(device, dtype=torch.long)


    outputs = model(ids, mask, token_type_ids)
    one_hot_labels = torch.nn.functional.one_hot(labels,num_classes=6).type_as(outputs)
    loss = loss_func(outputs, one_hot_labels)
    loss_train_total += loss.item()
    loss.backward()

    torch.nn.utils.clip_grad_norm_(model.parameters(),1.0)

    optimizer.step()
    if schedular is not None:
      schedular.step()

    progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
    
  loss_train_avg = loss_train_total/len(dataloader_train)
  return loss_train_avg

def evaluate(model, dataloader, device,epoch):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []

    progress_bar = tqdm(dataloader, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:
        
        ids = batch['ids']
        mask = batch['mask']
        token_type_ids = batch['token_type_ids']
        labels = batch['targets']
        
        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        labels = labels.to(device, dtype=torch.long)

        

        with torch.no_grad():        
            outputs = model(ids, mask, token_type_ids)
            
        one_hot_labels = torch.nn.functional.one_hot(labels,num_classes=6).type_as(outputs)
        loss = loss_func(outputs, one_hot_labels)
        logits = outputs
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = labels.cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
        
    loss_val_avg = loss_val_total/len(dataloader) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [ ]:
model_save_path = '/content/drive/My Drive/chatbot/models/'

In [ ]:
pretrained_model_path = '/content/drive/My Drive/chatbot/models/finetuned_BERT_epoch_2.pt'

In [ ]:
def get_class_weights(df_train):
    
    class_counts = [len(df_train[df_train['label'] == i]) for i in range(len(labels)) ]
    class_weights = 1 / torch.tensor(class_counts, dtype = torch.float)
    target_list = df_train.label.values
    class_weights_all = class_weights[target_list]
    
    return class_weights_all

In [ ]:
def run():
    print("entered in run")
 
    max_len = 512
    batch_size = 8
    epochs = 3
    tokenizer = BertTokenizer.from_pretrained('bert-base-cased', 
                                          do_lower_case=True)
    dataset_train = BERTDataset(
                      df_train.text_a.values,
                      df_train.text_b.values,
                      df_train.label.values,
                      tokenizer,
                      max_len = max_len )

    dataset_val = BERTDataset(
                      df_val.text_a.values,
                      df_val.text_b.values,
                      df_val.label.values,
                      tokenizer,
                      max_len = max_len )
    
    dataset_test = BERTDataset(
                      df_test.text_a.values,
                      df_test.text_b.values,
                      df_test.label.values,
                      tokenizer,
                      max_len = max_len )
    
    class_weights_all = get_class_weights(df_train)
    # print(len(class_weights_all))
    weighted_sampler = WeightedRandomSampler(
        weights=class_weights_all,
        num_samples=len(class_weights_all),
        replacement=True
    )
    
    dataloader_train = DataLoader(dataset_train, 
                                  sampler=weighted_sampler, 
                                  batch_size=batch_size)

    dataloader_validation = DataLoader(dataset_val, 
                                      sampler=SequentialSampler(dataset_val), 
                                      batch_size=batch_size)
    
    dataloader_test = DataLoader(dataset_test, 
                                      sampler=SequentialSampler(dataset_val), 
                                      batch_size=batch_size)
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(device)
    #init model
    model = BERTModel()

    load_model = True
    if load_model:
      model.load_state_dict(torch.load(pretrained_model_path, map_location=torch.device('cpu')))

    model.to(device)
    print("model loaded")
    #define optimizer
    optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
    
    #define schedular
    num_train_steps = int(len(dataset_train)/batch_size*epochs)
    schedular = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=num_train_steps)
    
    for epoch in tqdm(range(3,epochs+3)):
        
        #model training function
        loss_train_avg = train_fn(model,dataloader_train,optimizer,schedular,device,epoch)
        
        torch.save(model.state_dict(), model_save_path + f'finetuned_BERT_epoch_{epoch}.pt')
        
        tqdm.write(f'\nEpoch {epoch}')
        tqdm.write(f'Training loss: {loss_train_avg}')
        
        #evaluate val set
        val_loss, predictions, true_vals = evaluate(model, dataloader_test, device,epoch)
        
        #f1_score
        val_f1 = f1_score_func(predictions, true_vals)
        
        #print metrics
        tqdm.write(f'Validation loss: {val_loss}')
        tqdm.write(f'F1 Score (Weighted): {val_f1}')
        print('-------------X------------------------------------X------------')
        try:
          print(accuracy_per_class(predictions, true_vals))
        except Exception as e:
          print("error: ",e)

In [ ]:
run()

entered in run
cuda
model loaded



Epoch 3
Training loss: 0.14321579887989921


Validation loss: 0.18098148004189982
F1 Score (Weighted): 0.8623503755019858
-------------X------------------------------------X------------
Accuracy: 0.8584070796460177
F1 Score: 0.8623503755019858
Class: disagree
Class accuracy: 26/355,0.07323943661971831

Class: agree
Class accuracy: 618/1006,0.614314115308151

Class: discuss
Class accuracy: 1662/2475,0.6715151515151515

Class: unrelated
Class accuracy: 9819/10289,0.954320147730586

None



Epoch 4
Training loss: 0.10907879056834814


Validation loss: 0.18970595484016473
F1 Score (Weighted): 0.864600489768171
-------------X------------------------------------X------------
Accuracy: 0.8632920353982301
F1 Score: 0.864600489768171
Class: disagree
Class accuracy: 21/355,0.059154929577464786

Class: agree
Class accuracy: 599/1006,0.595427435387674

Class: discuss
Class accuracy: 1636/2475,0.661010101010101

Class: unrelated
Class accuracy: 9938/10289,0.9658858975605015

None



Epoch 5
Training loss: 0.08630987016721689


Validation loss: 0.20362789122282163
F1 Score (Weighted): 0.8674178365494415
-------------X------------------------------------X------------
Accuracy: 0.8658407079646018
F1 Score: 0.8674178365494415
Class: disagree
Class accuracy: 27/355,0.07605633802816901

Class: agree
Class accuracy: 576/1006,0.5725646123260437

Class: discuss
Class accuracy: 1668/2475,0.673939393939394

Class: unrelated
Class accuracy: 9959/10289,0.967926912236369

None



In [ ]:
run()

entered in run
cuda



model loaded



Epoch 1
Training loss: 0.14517673482113927


Validation loss: 0.10233342449547563
F1 Score (Weighted): 0.8690424569818683
-------------X------------------------------------X------------
Accuracy: 0.8781592920353982
F1 Score: 0.8690424569818683
Class: disagree
Class accuracy: 865/1296,0.6674382716049383

Class: agree
Class accuracy: 485/1419,0.34178999295278367

Class: discuss
Class accuracy: 1704/1956,0.8711656441717791

Class: unrelated
Class accuracy: 9350/9454,0.9889993653480008

None



Epoch 2
Training loss: 0.10741750664706441


Validation loss: 0.08799194261336901
F1 Score (Weighted): 0.8908237370420098
-------------X------------------------------------X------------
Accuracy: 0.896
F1 Score: 0.8908237370420098
Class: disagree
Class accuracy: 968/1296,0.7469135802469136

Class: agree
Class accuracy: 631/1419,0.44467935165609584

Class: discuss
Class accuracy: 1698/1956,0.8680981595092024

Class: unrelated
Class accuracy: 9359/9454,0.9899513433467315

None


Buffered data was truncated after reaching the output size limit.

In [ ]:
model = BERTModel()

load_model = True
if load_model:
  model.load_state_dict(torch.load(pretrained_model_path, map_location=torch.device('cpu')))

In [ ]:
torch.save(model,model_save_path + 'finetuned_BERT_epoch_2.bin')

In [ ]:
run()

entered in run
len of test data:  30239
cuda
model loaded



Epoch 1


Validation loss: 0.12454752876958045
F1 Score (Weighted): 0.8729178921284906
-------------X------------------------------------X------------
Accuracy: 0.8763893805309735
F1 Score: 0.8729178921284906
Class: disagree
Class accuracy: 12/355,0.03380281690140845

Class: agree
Class accuracy: 520/1006,0.5168986083499006

Class: discuss
Class accuracy: 1805/2475,0.7292929292929293

Class: unrelated
Class accuracy: 10042/10289,0.9759937797647974

None

